# Advanced techniques

First we need to activate minzinc Jupyter extension

In [1]:
%load_ext iminizinc

<IPython.core.display.Javascript object>

MiniZinc to FlatZinc converter, version 2.6.2, build 497830444
Copyright (C) 2014-2022 Monash University, NICTA, Data61


## Global constraints

Global constraints are constraints that handle more than 2 variables and due this are able to reduce domains better. Some well known global constraints:

* all_different
* element
* count
* cumulative
* table
  

There is [global constraints catalogue](https://sofdem.github.io/gccat/gccat/theindex.html).

### N-Queens problem

N-Queens problem is a generalization of [8 queens problem](https://en.wikipedia.org/wiki/Eight_queens_puzzle). This is the problem of placing eight chess queens on an 8×8 chessboard so that no two queens threaten each other.

![](images/8_queens_example.png)

### Solution without global constraints

In [ ]:
%%minizinc
% 
% n queens problem in MiniZinc.
% 
% This MiniZinc model was created by Hakan Kjellerstrand, hakank@bonetmail.com
% See also my MiniZinc page: http://www.hakank.org/minizinc
%
include "globals.mzn";

int: n;
array[1..n] of var 1..n: queens;

solve :: int_search(
        queens, 
        first_fail, 
        indomain_min, 
        complete) 
    satisfy;

% solve satisfy;

constraint
    forall(i, j in 1..n where i < j) (
         queens[i] != queens[j] /\
         queens[i] + i != queens[j] + j /\
         queens[i] - i != queens[j] - j
    ) 
 ;

output [
  show(queens) ++ "\n"

];

% data
n = 1000;

### Solution using global constraints

In [ ]:
%%minizinc
% 
% n queens problem in MiniZinc.
% 
% Using alldifferent. 
%
% 
% This MiniZinc model was created by Hakan Kjellerstrand, hakank@bonetmail.com
% See also my MiniZinc page: http://www.hakank.org/minizinc
%
include "globals.mzn";

int: n;
array[1..n] of var 1..n: queens;


solve :: int_search(
        queens, 
        first_fail,
        indomain_median,
        complete
       ) 
     satisfy;

% solve satisfy;

% Chuffed (with --free) seems to perform better without :: domain
% on the first constraint.
constraint all_different(queens); % :: domain;
constraint all_different([queens[i]+i | i in 1..n]); % :: domain;
constraint all_different([queens[i]-i | i in 1..n]); % :: domain;

output 
[
  show(queens)
] 
% ++ 
% [
%   if j = 1 then "\n" else "" endif ++
%      if fix(queens[i]) = j then          
%         % show_int(2,j)
%         "Q"
%      else
%         "#"
%      endif
%   | i in 1..n, j in 1..n
% ] ++
% ["\n"]
;

% data
n = 1000;

### Comparison

Using global *all_different* constraint lead to 10x speedup (28s to 2.8s).

## Search strategy

### Variable selection strategy

I will compare three strategies:

* *input_order*
* *smallest*
* *first_fail*

#### *input_order* strategy

In [ ]:
%%minizinc
% 
% n queens problem in MiniZinc.
% 
% Using alldifferent. 
%
% 
% This MiniZinc model was created by Hakan Kjellerstrand, hakank@bonetmail.com
% See also my MiniZinc page: http://www.hakank.org/minizinc
%
include "globals.mzn";

int: n;
array[1..n] of var 1..n: queens;


solve :: int_search(
        queens, 
        input_order,
        indomain_median,
        complete
       ) 
     satisfy;

% solve satisfy;

% Chuffed (with --free) seems to perform better without :: domain
% on the first constraint.
constraint all_different(queens); % :: domain;
constraint all_different([queens[i]+i | i in 1..n]); % :: domain;
constraint all_different([queens[i]-i | i in 1..n]); % :: domain;

output 
[
  show(queens)
] 
% ++ 
% [
%   if j = 1 then "\n" else "" endif ++
%      if fix(queens[i]) = j then          
%         % show_int(2,j)
%         "Q"
%      else
%         "#"
%      endif
%   | i in 1..n, j in 1..n
% ] ++
% ["\n"]
;

% data
n = 1000;

#### *smallest* strategy

In [ ]:
%%minizinc
% 
% n queens problem in MiniZinc.
% 
% Using alldifferent. 
%
% 
% This MiniZinc model was created by Hakan Kjellerstrand, hakank@bonetmail.com
% See also my MiniZinc page: http://www.hakank.org/minizinc
%
include "globals.mzn";

int: n;
array[1..n] of var 1..n: queens;


solve :: int_search(
        queens, 
        smallest,
        indomain_median,
        complete
       ) 
     satisfy;

% solve satisfy;

% Chuffed (with --free) seems to perform better without :: domain
% on the first constraint.
constraint all_different(queens); % :: domain;
constraint all_different([queens[i]+i | i in 1..n]); % :: domain;
constraint all_different([queens[i]-i | i in 1..n]); % :: domain;

output 
[
  show(queens)
] 
% ++ 
% [
%   if j = 1 then "\n" else "" endif ++
%      if fix(queens[i]) = j then          
%         % show_int(2,j)
%         "Q"
%      else
%         "#"
%      endif
%   | i in 1..n, j in 1..n
% ] ++
% ["\n"]
;

% data
n = 1000;

#### *first-fail* strategy

In [ ]:
%%minizinc
% 
% n queens problem in MiniZinc.
% 
% Using alldifferent. 
%
% 
% This MiniZinc model was created by Hakan Kjellerstrand, hakank@bonetmail.com
% See also my MiniZinc page: http://www.hakank.org/minizinc
%
include "globals.mzn";

int: n;
array[1..n] of var 1..n: queens;


solve :: int_search(
        queens, 
        first_fail,
        indomain_median,
        complete
       ) 
     satisfy;

% solve satisfy;

% Chuffed (with --free) seems to perform better without :: domain
% on the first constraint.
constraint all_different(queens); % :: domain;
constraint all_different([queens[i]+i | i in 1..n]); % :: domain;
constraint all_different([queens[i]-i | i in 1..n]); % :: domain;

output 
[
  show(queens)
] 
% ++ 
% [
%   if j = 1 then "\n" else "" endif ++
%      if fix(queens[i]) = j then          
%         % show_int(2,j)
%         "Q"
%      else
%         "#"
%      endif
%   | i in 1..n, j in 1..n
% ] ++
% ["\n"]
;

% data
n = 1000;

#### Comparison summary

|Strategy   | Time   |
|---|---|
|input_order | Inf |
|smallest | Inf  |
|first-fail   | 2.8s  |

### Value selection strategy

I will keep *first-fail* strategy and consider following value selection strategies:

* indomain_min
* indomain_random
* indomain_median
* indomain_split

#### indomain_min

In [ ]:
%%minizinc
% 
% n queens problem in MiniZinc.
% 
% Using alldifferent. 
%
% 
% This MiniZinc model was created by Hakan Kjellerstrand, hakank@bonetmail.com
% See also my MiniZinc page: http://www.hakank.org/minizinc
%
include "globals.mzn";

int: n;
array[1..n] of var 1..n: queens;


solve :: int_search(
        queens, 
        first_fail,
        indomain_min,
        complete
       ) 
     satisfy;

% solve satisfy;

% Chuffed (with --free) seems to perform better without :: domain
% on the first constraint.
constraint all_different(queens); % :: domain;
constraint all_different([queens[i]+i | i in 1..n]); % :: domain;
constraint all_different([queens[i]-i | i in 1..n]); % :: domain;

output 
[
  show(queens)
] 
% ++ 
% [
%   if j = 1 then "\n" else "" endif ++
%      if fix(queens[i]) = j then          
%         % show_int(2,j)
%         "Q"
%      else
%         "#"
%      endif
%   | i in 1..n, j in 1..n
% ] ++
% ["\n"]
;

% data
n = 1000;

#### indomain_random

In [ ]:
%%minizinc
% 
% n queens problem in MiniZinc.
% 
% Using alldifferent. 
%
% 
% This MiniZinc model was created by Hakan Kjellerstrand, hakank@bonetmail.com
% See also my MiniZinc page: http://www.hakank.org/minizinc
%
include "globals.mzn";

int: n;
array[1..n] of var 1..n: queens;


solve :: int_search(
        queens, 
        first_fail,
        indomain_random,
        complete
       ) 
     satisfy;

% solve satisfy;

% Chuffed (with --free) seems to perform better without :: domain
% on the first constraint.
constraint all_different(queens); % :: domain;
constraint all_different([queens[i]+i | i in 1..n]); % :: domain;
constraint all_different([queens[i]-i | i in 1..n]); % :: domain;

output 
[
  show(queens)
] 
% ++ 
% [
%   if j = 1 then "\n" else "" endif ++
%      if fix(queens[i]) = j then          
%         % show_int(2,j)
%         "Q"
%      else
%         "#"
%      endif
%   | i in 1..n, j in 1..n
% ] ++
% ["\n"]
;

% data
n = 1000;

#### indomain_split

In [ ]:
%%minizinc
% 
% n queens problem in MiniZinc.
% 
% Using alldifferent. 
%
% 
% This MiniZinc model was created by Hakan Kjellerstrand, hakank@bonetmail.com
% See also my MiniZinc page: http://www.hakank.org/minizinc
%
include "globals.mzn";

int: n;
array[1..n] of var 1..n: queens;


solve :: int_search(
        queens, 
        first_fail,
        indomain_split,
        complete
       ) 
     satisfy;

% solve satisfy;

% Chuffed (with --free) seems to perform better without :: domain
% on the first constraint.
constraint all_different(queens); % :: domain;
constraint all_different([queens[i]+i | i in 1..n]); % :: domain;
constraint all_different([queens[i]-i | i in 1..n]); % :: domain;

output 
[
  show(queens)
] 
% ++ 
% [
%   if j = 1 then "\n" else "" endif ++
%      if fix(queens[i]) = j then          
%         % show_int(2,j)
%         "Q"
%      else
%         "#"
%      endif
%   | i in 1..n, j in 1..n
% ] ++
% ["\n"]
;

% data
n = 1000;

#### Comparison summary

| Strategy | Time |
|----------|------|
|indomain_min| 2.1s|
|first_fail|2.8s|
|indomain_random|5.2s|

### Custom strategy

#### Jobshop scheduling problem

[Following Wikipedia](https://en.wikipedia.org/wiki/Job-shop_scheduling)

> Job-shop scheduling or the job-shop problem (JSP) is an optimization problem in computer science and operations research. It is a |variant of optimal job scheduling. In a general job scheduling problem, we are given n jobs J1, J2, ..., Jn of varying processing times, which need to be scheduled on m machines with varying processing power, while trying to minimize the makespan – the total length of the schedule (that is, when all the jobs have finished processing). In the specific variant known as job-shop scheduling, each job consists of a set of operations O1, O2, ..., On which need to be processed in a specific order (known as precedence constraints). Each operation has a specific machine that it needs to be processed on and only one operation in a job can be processed at a given time. A common relaxation is the flexible job shop, where each operation can be processed on any machine of a given set (the machines in each set are identical).

![](images/jobshop_example.png)

Source: [https://www.kecl.ntt.co.jp/as/members/yamada/galbk.pdf](https://www.kecl.ntt.co.jp/as/members/yamada/galbk.pdf)

##### First version

First search strategy is straighforward:

* Find task with smallest value in its starting time domain (the one we can schedule earliest)
* Set starting to minimum value

In [13]:
%%minizinc?

Docstring:
::

  %minizinc [-v] [-s] [-m {return,bind}] [-a] [-t TIME_LIMIT]
                [--solver SOLVER] [--data [DATA ...]]
                [model ...]

MiniZinc magic

positional arguments:
  model                 Model to solve

optional arguments:
  -v, --verbose         Verbose output
  -s, --statistics      Output statistics
  -m <{return,bind}>, --solution-mode <{return,bind}>
                        Whether to return solution(s) or bind them to
                        variables
  -a, --all-solutions   Return all solutions for satisfaction problems,
                        intermediate solutions for optimisation problems.
                        Implies -o.
  -t TIME_LIMIT, --time-limit TIME_LIMIT
                        Time limit in milliseconds (includes compilation and
                        solving)
  --solver SOLVER       Solver to run
  --data <[DATA ...]>   Data files
File:      ~/projects/conf-wdi2022-slaying-dragons-with-constraint-programming/python/venv/lib/py

In [2]:
experiments = {}

In [15]:
%%minizinc -m bind -s -t 60000 --solver gecode

include "disjunctive.mzn";

int: n_machines;                        % The number of machines.
int: n_jobs;                            % The number of jobs.
int: n_tasks = n_machines;      % Each job has one task per machine.
set of int: jobs = 1..n_jobs;
set of int: tasks = 1..n_tasks;
set of int: machines = 0..(n_machines-1);

array [jobs, tasks] of int: jt_machine;
array [jobs, tasks] of int: jt_duration;

int: max_end;

array [jobs, tasks] of var 0..max_end: jt_start;
var 0..max_end: t_end = max([jt_start[j,t] + jt_duration[j,t] | j in jobs, t in tasks]);


constraint
    (
        t_end = max([jt_start[j,t] + jt_duration[j,t] | j in jobs, t in tasks])
    );
constraint
    forall ( j in jobs, k in 1..(n_tasks - 1) ) (
        jt_start[j, k] + jt_duration[j, k]  <=
            jt_start[j, k + 1]
    );

constraint
    forall(m in machines) (
        disjunctive(
            [jt_start[j,t] | j in jobs, t in tasks where
                jt_machine[j,t]=m],
            [jt_duration[j,t] | j in jobs, t in tasks where
                jt_machine[j,t]=m])
    );
solve 
    ::
    int_search([jt_start[j,t] | j in jobs, t in tasks],
                smallest,
                indomain_min)
    minimize t_end;

n_machines = 10;
n_jobs = 10;
max_end = 1050;


%
% The times for each job.
%
jt_duration = array2d(jobs, tasks, 
   [
 29, 78, 9,  36, 49, 11, 62, 56, 44, 21,
 43, 90, 75, 11, 69, 28, 46, 46, 72, 30,
 91, 85, 39, 74, 90, 10, 12, 89, 45, 33,
 81, 95, 71, 99, 9,  52, 85, 98, 22, 43,
 14, 6,  22, 61, 26, 69, 21, 49, 72, 53,
 84, 2,  52, 95, 48, 72, 47, 65, 6,  25,
 46, 37, 61, 13, 32, 21, 32, 89, 30, 55,
 31, 86, 46, 74, 32, 88, 19, 48, 36, 79,
 76, 69, 76, 51, 85, 11, 40, 89, 26, 74,
 85, 13, 61, 7,  64, 76, 47, 52, 90, 45,
 ]);


%
% The order the jobs must be done.
%
%
jt_machine = array2d(jobs, 1..n_machines,
  [
   % indicating the order in which each job must be done
 0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
 0, 2, 4, 9, 3, 1, 6, 5, 7, 8,
 1, 0, 3, 2, 8, 5, 7, 6, 9, 4,
 1, 2, 0, 4, 6, 8, 7, 3, 9, 5,
 2, 0, 1, 5, 3, 4, 8, 7, 9, 6,
 2, 1, 5, 3, 8, 9, 0, 6, 4, 7,
 1, 0, 3, 2, 6, 5, 9, 8, 7, 4,
 2, 0, 1, 5, 4, 6, 8, 9, 7, 3,
 0, 1, 3, 5, 2, 9, 6, 7, 4, 8,
 1, 0, 2, 6, 8, 9, 5, 3, 4, 7,
  ]);

MiniZinc to FlatZinc converter, version 2.6.2, build 497830444
Copyright (C) 2014-2022 Monash University, NICTA, Data61
Parsing file(s) /var/folders/1d/g431hzp941z_qmtsghb8qgb00000gn/T/tmpnflb9jnz/model.mzn' ...
processing file '/Applications/MiniZincIDE.app/Contents/Resources/share/minizinc/std/stdlib.mzn'
processing file '/Applications/MiniZincIDE.app/Contents/Resources/share/minizinc/std/stdlib/stdlib_internal.mzn'
processing file '/Applications/MiniZincIDE.app/Contents/Resources/share/minizinc/std/output.mzn'
processing file '/Applications/MiniZincIDE.app/Contents/Resources/share/minizinc/std/output/array2d_bool.mzn'
processing file '/Applications/MiniZincIDE.app/Contents/Resources/share/minizinc/std/output/gantt.mzn'
processing file '/Applications/MiniZincIDE.app/Contents/Resources/share/minizinc/std/stdlib/stdlib_random.mzn'
processing file '/Applications/MiniZincIDE.app/Contents/Resources/share/minizinc/std/stdlib/stdlib_debug.mzn'
processing file '/Applications/MiniZincIDE.app/

[{'jt_start': [[0, 91, 188, 197, 263, 333, 344, 406, 509, 572],
   [29, 98, 188, 263, 278, 389, 417, 463, 509, 647],
   [0, 154, 239, 345, 419, 509, 581, 593, 723, 768],
   [169, 250, 362, 442, 541, 677, 729, 822, 971, 993],
   [0, 72, 250, 272, 347, 373, 553, 593, 899, 971],
   [14, 272, 344, 434, 729, 779, 851, 898, 963, 974],
   [274, 325, 373, 434, 463, 519, 540, 796, 944, 974],
   [197, 239, 502, 548, 622, 682, 777, 851, 908, 944],
   [78, 320, 529, 622, 673, 768, 779, 819, 912, 938],
   [417, 502, 515, 576, 583, 647, 723, 770, 822, 999]]}]

In [23]:
import nest_asyncio
nest_asyncio.apply()

import datetime

In [37]:
from minizinc import Instance, Model, Solver 

jobshop_1 = Model("./jobshop_1.mzn")
gecode = Solver.lookup("gecode")
# Create an Instance of the n-Queens model for Gecode
instance = Instance(gecode, jobshop_1)
result = instance.solve(timeout=datetime.timedelta(minutes=1))
experiments["first_version"]=result.solution.objective

In [30]:
result.solution.objective

1044

##### Second version

Here we will take a different strategy of searching for a solution:

* We add sequence variables that say in what order jobs are executed at each machine. 
* We have the following search strategy:
    * Find sequence var with smallest domain
    * Select random value for order variable
    * Find task with smallest value in its starting time domain (the one we can schedule earliest)
    * Set starting to minimum value

In [ ]:
%%minizinc -m bind -s -t 60000 --solver gecode

include "globals.mzn";

int: n_machines;                        % The number of machines.
int: n_jobs;                            % The number of jobs.
int: n_tasks = n_machines;      % Each job has one task per machine.
set of int: jobs = 1..n_jobs;
set of int: tasks = 1..n_tasks;
set of int: machines = 0..(n_machines-1);

array [jobs, tasks] of int: jt_machine;
array [jobs, tasks] of int: jt_duration;

int: max_end;

array [jobs, tasks] of var 0..max_end: jt_start;
var 0..max_end: t_end;

array [machines, jobs] of var jobs: seq;


constraint
    (
        t_end = max([jt_start[j,t] + jt_duration[j,t] | j in jobs, t in tasks])
    );

constraint
    forall ( j in jobs, k in 1..(n_tasks - 1) ) (
        jt_start[j, k] + jt_duration[j, k]  <=
            jt_start[j, k + 1]
    );

constraint
    forall(m in machines) (
        disjunctive(
            [jt_start[j,t] | j in jobs, t in tasks where
                jt_machine[j,t]=m],
            [jt_duration[j,t] | j in jobs, t in tasks where
                jt_machine[j,t]=m])
    );

constraint
    forall(m in machines) (alldifferent([seq[m,j] | j in jobs]));

constraint
    forall(m in machines, j1 in jobs, j2 in jobs, t1 in tasks, t2 in tasks
        where jt_machine[j1,t1]=m /\jt_machine[j2,t2]=m)
        (
            seq[m, j1] < seq[m, j2] -> 
                jt_start[j1,t1] + jt_duration[j1, t1] <= jt_start[j2, t2]            
        );

solve 
    :: seq_search([
        int_search(
            [seq[m,j] | 
                m in machines, t in tasks, j in jobs
                where jt_machine[j,t]=m],
            first_fail,
            indomain_random),
        int_search([jt_start[j,t] | j in jobs, t in tasks], smallest, indomain_min)])
    :: restart_constant(1000)

    minimize t_end;

output [
    "t_end = ", show(t_end), "\n"
];


n_machines = 10;
n_jobs = 10;
max_end = 1050;


%
% The times for each job.
%
jt_duration = array2d(jobs, tasks, 
   [
 29, 78, 9,  36, 49, 11, 62, 56, 44, 21,
 43, 90, 75, 11, 69, 28, 46, 46, 72, 30,
 91, 85, 39, 74, 90, 10, 12, 89, 45, 33,
 81, 95, 71, 99, 9,  52, 85, 98, 22, 43,
 14, 6,  22, 61, 26, 69, 21, 49, 72, 53,
 84, 2,  52, 95, 48, 72, 47, 65, 6,  25,
 46, 37, 61, 13, 32, 21, 32, 89, 30, 55,
 31, 86, 46, 74, 32, 88, 19, 48, 36, 79,
 76, 69, 76, 51, 85, 11, 40, 89, 26, 74,
 85, 13, 61, 7,  64, 76, 47, 52, 90, 45,
 ]);


%
% The order the jobs must be done.
%
%
jt_machine = array2d(jobs, 1..n_machines,
  [
   % indicating the order in which each job must be done
 0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
 0, 2, 4, 9, 3, 1, 6, 5, 7, 8,
 1, 0, 3, 2, 8, 5, 7, 6, 9, 4,
 1, 2, 0, 4, 6, 8, 7, 3, 9, 5,
 2, 0, 1, 5, 3, 4, 8, 7, 9, 6,
 2, 1, 5, 3, 8, 9, 0, 6, 4, 7,
 1, 0, 3, 2, 6, 5, 9, 8, 7, 4,
 2, 0, 1, 5, 4, 6, 8, 9, 7, 3,
 0, 1, 3, 5, 2, 9, 6, 7, 4, 8,
 1, 0, 2, 6, 8, 9, 5, 3, 4, 7,
  ]);

In [34]:
jobshop_2 = Model("./jobshop_2.mzn")
gecode = Solver.lookup("gecode")
# Create an Instance of the n-Queens model for Gecode
instance = Instance(gecode, jobshop_2)
result = instance.solve(timeout=datetime.timedelta(minutes=1))
experiments["second_version"]=result.solution.objective

In [35]:
result.solution.objective

938

##### Comparison summary

In [36]:
print(experiments)

{'second_version': 938}
